In [3]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)

StatementMeta(, 0ed3e3fc-7f4b-44e3-b27c-061609f8c1a8, 5, Finished, Available)

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


In [4]:
df.write.format("delta").saveAsTable("raw_nyc_yellow_taxi_trips")

StatementMeta(, 0ed3e3fc-7f4b-44e3-b27c-061609f8c1a8, 6, Finished, Available)

In [41]:
import pyspark.sql.functions as F
df = (
  spark.table("raw_nyc_yellow_taxi_trips")
    .where(""" 
        (startLon between -74.4 and -73.6) AND
        (startLat between 40 and 41) AND
        (endLon between -74.4 and -73.6) AND
        (endLat between 40 and 41)
      """
    )
    .repartition(64)
)

df.write.format("delta").mode("overwrite").save("Files/nyc-yellow-taxi/plain-trips")

StatementMeta(, 0ed3e3fc-7f4b-44e3-b27c-061609f8c1a8, 43, Finished, Available)

In [ ]:
if mssparkutils.fs.exists("Files/nyc-yellow-taxi"):
    mssparkutils.fs.rm("Files/nyc-yellow-taxi", True)

In [10]:
%%sql
SELECT count(*) FROM delta.`Files/nyc-yellow-taxi/plain-trips` 

StatementMeta(, 437fb603-499c-45e2-bbce-5abc127cafce, 12, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [8]:
(
spark
    .sql("SELECT * FROM delta.`Files/nyc-yellow-taxi/plain-trips`")
    .write
    .format("delta")
    .mode("overwrite")
    .save("Files/nyc-yellow-taxi/optimised-trips")
)

StatementMeta(, 437fb603-499c-45e2-bbce-5abc127cafce, 10, Finished, Available)

In [9]:
from delta import DeltaTable

# Set OPTIMIZE max file size to 256MB
spark.conf.get("spark.databricks.delta.optimize.maxFileSize", "268435456")

table = DeltaTable.forPath(spark, "Files/nyc-yellow-taxi/optimised-trips")

display(table.optimize().executeZOrderBy("startLon", "startLat"))

StatementMeta(, 437fb603-499c-45e2-bbce-5abc127cafce, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9b14cf91-dc7c-4998-b9fd-0d8829b3524f)